In [255]:
import numpy as np
from scipy.spatial.transform import Rotation

### Setup

In [256]:
sigma_min, sigma_max = 0.9, 1.1

In [257]:
initial_positions = np.array(
    [[1.0, 0.0, 0.0], [3.0, 1.0, 0.0], [1.0, 3.0, 3.0], [2.0, 2.0, 2.0]]
)

In [258]:
A = np.array(
    [
        [1, 0, 0, -1],
        [0, 1, 0, -1],
        [0, 0, 1, -1],
    ]
)

In [259]:
S = np.zeros((4, 4))
S[0, 0] = 1
S[1, 1] = 1
S[2, 2] = 1
S[3, 3] = 1

In [260]:
X_g = (A @ S @ initial_positions).T
X_g_inv = np.linalg.pinv(X_g)

In [261]:
X_g @ X_g_inv

array([[ 1.00000000e+00, -8.88178420e-16,  5.55111512e-16],
       [ 1.11022302e-16,  1.00000000e+00,  7.77156117e-16],
       [ 0.00000000e+00,  4.44089210e-16,  1.00000000e+00]])

### Step

In [262]:
rotation = Rotation.from_euler("xy", [30, 50], degrees=True)
current_positions = rotation.apply(initial_positions)

In [263]:
X_f = (A @ S @ current_positions).T

In [264]:
U, sigma, V_t = np.linalg.svd(X_f @ X_g_inv)

In [265]:
sigma = np.diag(sigma)

In [271]:
U @ sigma @ V_t

array([[ 6.42787610e-01,  3.83022222e-01,  6.63413948e-01],
       [-1.13987598e-16,  8.66025404e-01, -5.00000000e-01],
       [-7.66044443e-01,  3.21393805e-01,  5.56670399e-01]])

In [272]:
(sigma @ X_g).T

array([[-1., -2., -2.],
       [ 1., -1., -2.],
       [-1.,  1.,  1.]])

In [246]:
def C(D):
    det_Sigma = np.linalg.det(sigma + np.diag(D))
    return max(det_Sigma - sigma_max, det_Sigma - sigma_min)

In [247]:
def DC(D):
    det_Sigma = np.repeat(np.linalg.det(sigma + np.diag(D)), 3)
    div = (np.diagonal(sigma) + D)
    return det_Sigma / np.maximum(div, 0.000001)

In [248]:
def get_D_k(D):
    return (DC(D).T @ D - C(D)) / np.maximum(np.dot(DC(D), DC(D)), 0.00001) * DC(D)

In [249]:
D_0 = np.random.random(3)
D_k = D_0.copy()

for _ in range(100):
    sigma_p = sigma + np.diag(D_k)
    det_sigma_p = np.linalg.det(sigma_p)
    
    DC_D = np.repeat(det_sigma_p, 3) / np.maximum(np.diagonal(sigma_p), 0.000001)
    C_D = max(det_sigma_p - sigma_max, det_sigma_p - sigma_min)
    
    D_k = (DC_D.T @ D_k - C_D) / np.maximum(np.dot(DC_D, DC_D), 0.00001) * DC_D

In [254]:
np.linalg.det(sigma_p)

0.9

In [251]:
DC(D_k)

array([0.93216975, 0.93216975, 0.93216975])

In [252]:
D_0

array([0.12753648, 0.91528047, 0.45833766])